In [1]:
from math import nan
from future.utils import iteritems
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
data = pd.read_csv("ner_dataset.csv", encoding= 'latin1') 
data = data.fillna(method="ffill")
data.head(672)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
667,Sentence: 29,Muslim,NNP,B-org
668,Sentence: 29,Brotherhood,NNP,I-org
669,Sentence: 29,as,IN,O
670,Sentence: 29,parts,NNS,O


In [18]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
nrofWords = len(words)

ner_tags = list(set(data["Tag"].values))
nrofTags = len(ner_tags)
print(nrofWords, nrofTags)
print(ner_tags)

35179 17
['B-org', 'B-gpe', 'O', 'I-geo', 'B-per', 'I-art', 'I-org', 'B-tim', 'I-per', 'I-tim', 'I-eve', 'I-nat', 'B-art', 'I-gpe', 'B-eve', 'B-nat', 'B-geo']


In [5]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

In [6]:
word_index = {w: i +1 for i, w in enumerate(words)}
tag_index = {t: i for i, t in enumerate(ner_tags)}
idx2tag = {v: k for k, v in iteritems(tag_index)}

In [7]:
word_index["universe"]

6064

In [8]:
maxlen = max([len(s) for s in sentences])
word_embedding_size = 17

X = [[word_index[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=nrofWords - 1)

y = [[tag_index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag_index["O"])
y = [to_categorical(i, num_classes=nrofTags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Simple RNN model

In [29]:
input = Input(shape=(maxlen,))

model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=20, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(32, return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])



In [30]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model_1.summary()

history_1 = model_1.fit(X_train, np.array(y_train), batch_size=32, epochs=10,
                    validation_split=0.2, verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 104, 20)           703600    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 104, 128)          10880     
_________________________________________________________________
dense_2 (Dense)              (None, 104, 17)           2193      
_________________________________________________________________
dropout_2 (Dropout)          (None, 104, 17)           0         
Total params: 716,673
Trainable params: 716,673
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
960/960 [==============================] - 49s 51ms/step - loss: 0.4316 - accuracy: 0.5957 - val_loss: 0.3189 - val_accuracy: 0.9680
Epoch 2/30
960/960 [==============================] - 41s 42ms/step - loss: 0.4142 - accuracy: 0.6065 

In [31]:
test_pred_1 = model_1.predict(X_test, verbose=1)

idx2tag = {i: w for w, i in tag_index.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred_1)
test_labels = pred2label(y_test)

300/300 [==============================] - 4s 12ms/step


In [32]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))

F1-score: 13.1%
           precision    recall  f1-score   support

      org       0.00      0.00      0.00      4087
      gpe       0.00      0.00      0.00      3190
      geo       0.00      0.00      0.00      7434
      tim       0.00      0.00      0.00      4133
      per       0.15      0.77      0.25      3364
      art       0.00      0.00      0.00        84
      eve       0.00      0.00      0.00        60
      nat       0.00      0.00      0.00        46

micro avg       0.15      0.12      0.13     22398
macro avg       0.02      0.12      0.04     22398



In [33]:
i = 1928
p = model_1.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], ner_tags[t], ner_tags[pred]))


Word           ||True ||Pred
The            : O     O
Bloomberg      : B-org B-per
financial      : O     O
news           : O     O
service        : O     O
reports        : O     O
Kenya          : B-geo B-per
received       : O     O
more           : O     O
than           : O     O
one            : O     O
million        : O     O
tourists       : O     O
in             : O     O
the            : O     O
first          : B-tim O
nine           : I-tim O
months         : O     O
of             : B-tim O
2007           : I-tim I-per
and            : O     O
is             : O     O
the            : O     O
largest        : O     O
exporter       : O     O
of             : O     O
black          : O     O
tea            : O     O
in             : O     O
the            : O     O
world          : O     O
.              : O     O
Raising        : O     O
Raising        : O     O
Raising        : O     O
Raising        : O     O
Raising        : O     O
Raising        : O     O
Raising  

# LSTM model

In [34]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=word_embedding_size, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])

In [35]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [36]:
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 104, 17)           598060    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 104, 128)          41984     
_________________________________________________________________
dense_3 (Dense)              (None, 104, 17)           2193      
_________________________________________________________________
dropout_3 (Dropout)          (None, 104, 17)           0         
Total params: 642,237
Trainable params: 642,237
Non-trainable params: 0
_________________________________________________________________


In [38]:
history_2 = model_2.fit(X_train, np.array(y_train), batch_size=32, epochs=5,
                    validation_split=0.5, verbose=1)


Epoch 1/5
600/600 [==============================] - 89s 148ms/step - loss: 0.4138 - accuracy: 0.6061 - val_loss: 0.3173 - val_accuracy: 0.9678
Epoch 2/5
600/600 [==============================] - 78s 130ms/step - loss: 0.4138 - accuracy: 0.6057 - val_loss: 0.3173 - val_accuracy: 0.9678
Epoch 3/5
600/600 [==============================] - 75s 126ms/step - loss: 0.4137 - accuracy: 0.6066 - val_loss: 0.3172 - val_accuracy: 0.9678
Epoch 4/5
600/600 [==============================] - 84s 140ms/step - loss: 0.4137 - accuracy: 0.6056 - val_loss: 0.3171 - val_accuracy: 0.9678
Epoch 5/5
600/600 [==============================] - 79s 131ms/step - loss: 0.4136 - accuracy: 0.6065 - val_loss: 0.3171 - val_accuracy: 0.9678


In [39]:
test_pred_2 = model_2.predict(X_test, verbose=1)

#idx2tag = {i: w for w, i in index_tag.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred_2)
test_labels = pred2label(y_test)


300/300 [==============================] - 7s 23ms/step


In [40]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 0.0%


In [41]:
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

      org       0.00      0.00      0.00      4087
      gpe       0.00      0.00      0.00      3190
      geo       0.00      0.00      0.00      7434
      tim       0.00      0.00      0.00      4133
      per       0.00      0.00      0.00      3364
      art       0.00      0.00      0.00        84
      eve       0.00      0.00      0.00        60
      nat       0.00      0.00      0.00        46

micro avg       0.00      0.00      0.00     22398
macro avg       0.00      0.00      0.00     22398



In [ ]:
i = 2007
p = model_1.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], ner_tags[t], ner_tags[pred]))
